# Low-level

---

## Kiểm tra môi trường

In [1]:
import sys
import os
sys.executable

'/usr/bin/python3'

## Import các thư viện cần thiết

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Row
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
import pandas as pd
import time
from math import sqrt


## Spark initialization

In [3]:
spark = SparkSession.builder \
    .appName("DecisionTreeRegressor") \
        .getOrCreate()
sc = spark.sparkContext

## Load the dataset

In [ ]:
import os
print(os.getcwd())
base_path = "/home/tienanh/22120017/Lab03_Spark/dataset/nyc-taxi-trip-duration"

/content


In [6]:
sample_data = sc.textFile(os.path.join(base_path, "sample_submission/sample_submission.csv"))
train_data = sc.textFile(os.path.join(base_path, "train/train.csv"))
test_data = sc.textFile(os.path.join(base_path, "test/test.csv"))

## Parse the dataset

In [7]:
sample_data_header = sample_data.first()
train_data_header = train_data.first()
test_data_header = test_data.first()

sample_data = sample_data.filter(lambda line: line != sample_data_header)
train_data = train_data.filter(lambda line: line != train_data_header)
test_data = test_data.filter(lambda line: line != test_data_header)

In [8]:
train_data.take(5)

['id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455',
 'id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663',
 'id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124',
 'id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429',
 'id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625,40.793209075927734,-73.972923278808594,40.782520294189453,N,435']

In [9]:
# Tiền xử lý: Chuyển mỗi dòng thành list các float
train_data_parsed = train_data.map(lambda line: line.split(",")).map(lambda parts: [
    float(parts[5]), float(parts[6]),  # pickup_longitude, pickup_latitude
    float(parts[7]), float(parts[8]),  # dropoff_longitude, dropoff_latitude
    float(parts[4]),                   # passenger_count
    float(parts[10])                   # trip_duration
])

## Pre-process the data

Chia tập dữ liệu thành 2 phần 80/20:
- `train`: Tập dữ liệu dùng để huấn luyện mô hình.
- `validation`: Tập dữ liệu dùng để đánh giá mô hình đã được traning trước đó.

In [10]:
train, validation = train_data_parsed.randomSplit([0.8, 0.2], seed=42)

## Train the model

### Split data
Chia dữ liệu thành 2 nhánh (trái, phải) dựa trên `feature_index` và `threshold`
- Nhánh trái chứa các điểm có giá trị ≤ threshold
- Nhánh phải chứa các điểm có giá trị > threshold  

In [11]:
def split_data(data, feature_index, threshold):
    left = data.filter(lambda x: x[feature_index] <= threshold)
    right = data.filter(lambda x: x[feature_index] > threshold)
    return left, right

### Calculate mse
- Mean Squared Error (MSE) của tập dữ liệu đầu vào và giá trị trung bình y. MSE được dùng để đo độ phân tán so với trung bình.

In [12]:
def calculate_mse(data):
    y_i = data.map(lambda x: x[-1])
    y_i_mean = y_i.mean()
    mse = y_i.map(lambda x: (x - y_i_mean) ** 2).mean()
    return mse, y_i_mean

### Find best split
- Tìm ra `feature` và ngưỡng `threshold` tốt nhất để chia dữ liệu, sao cho MSE của hai tập con (sau khi chia) là nhỏ nhất.

In [13]:

def find_best_split(data, num_features, sample_thresholds=20):
    best_feature, best_threshold, best_mse = None, None, float("inf")
    data_count = data.count()

    for feature_idx in range(num_features):
        # Lấy mẫu threshold đại diện
        feature_values = data.map(lambda x: x[feature_idx]).distinct().takeSample(False, sample_thresholds)
        for threshold in feature_values:
            left, right = split_data(data, feature_idx, threshold)
            left_count = left.count()
            right_count = right.count()

            # Bỏ qua nếu chia không hợp lý
            if left_count == 0 or right_count == 0:
                continue

            left_mse, _ = calculate_mse(left)
            right_mse, _ = calculate_mse(right)
            mse = (left_mse * left_count + right_mse * right_count) / data_count

            if mse < best_mse:
                best_feature, best_threshold, best_mse = feature_idx, threshold, mse
    return best_feature, best_threshold

### Build tree
Xây dựng cây theo phương pháp đệ quy:
- Nếu đã đạt đến độ sâu tối đa hoặc dữ liệu quá ít -> trả về giá trị trung bình hàm lá (dự đoán)
- Ngược lại:
    - Tìm cách chia tối ưu (find_best_split)
    - Đệ quy xây nhánh trái và phải

In [14]:

def build_tree(data, depth, max_depth):
    if depth == max_depth or data.count() <= 5:
        _, y_i_mean = calculate_mse(data)
        return y_i_mean

    best_feature, best_threshold = find_best_split(data, len(data.first()) - 1)

    if best_feature is None:
        _, y_i_mean = calculate_mse(data)
        return y_i_mean

    left, right = split_data(data, best_feature, best_threshold)
    left.cache()
    right.cache()

    return {
        "feature": best_feature,
        "threshold": best_threshold,
        "left": build_tree(left, depth + 1, max_depth),
        "right": build_tree(right, depth + 1, max_depth)
    }

In [15]:
# Training
start_time = time.time()
tree = build_tree(train, 0, 3)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Training time: 10661.048269748688 seconds


### Predict
Kiểm tra nếu tree là một dictionary -> đây là nút không phải lá.

So sánh row[feature] với threshold:
- Nếu nhỏ hơn hoặc bằng -> duyệt tiếp xuống nhánh trái (left)
- Ngược lại -> duyệt xuống nhánh phải (right)
- Nếu không phải dict (tức là giá trị thực tại lá) -> trả về giá trị đó (là dự đoán).

In [16]:
def predict(tree, row):
    if isinstance(tree, dict):
        if row[tree["feature"]] <= tree["threshold"]:
            return predict(tree["left"], row)
        else:
            return predict(tree["right"], row)
    else:
        return tree

##  Evaluate the model


In [17]:
# Evaluate on validation dataset
sample_test = validation.take(5)
for row in sample_test:
    print(f"Features: {row[:-1]}, Actual: {row[-1]}, Predicted: {predict(tree, row)}")

Features: [-73.98104858398438, 40.74433898925781, -73.9729995727539, 40.78998947143555, 1.0], Actual: 1225.0, Predicted: 824.643509031342
Features: [-73.99153137207031, 40.74943923950195, -73.95654296875, 40.7706298828125, 4.0], Actual: 1128.0, Predicted: 824.643509031342
Features: [-73.95551300048828, 40.768592834472656, -73.94876098632812, 40.77154541015625, 1.0], Actual: 211.0, Predicted: 824.643509031342
Features: [-73.98388671875, 40.738197326660156, -73.99120330810547, 40.72787094116211, 1.0], Actual: 251.0, Predicted: 824.643509031342
Features: [-73.97953796386719, 40.753360748291016, -73.96399688720703, 40.763458251953125, 1.0], Actual: 652.0, Predicted: 824.643509031342


In [20]:
predictions = validation.map(lambda row: (row[-1], predict(tree, row)))
mse = predictions.map(lambda x: (x[0] - x[1]) ** 2).mean()

rmse = sqrt(mse)

# Tính toán R² Score
mean_y_true = validation.map(lambda row: row[-1]).mean()
ss_total = validation.map(lambda row: (row[-1] - mean_y_true) ** 2).sum()
ss_residual = predictions.map(lambda x: (x[0] - x[1]) ** 2).sum()
r2 = 1 - (ss_residual / ss_total)

# In kết quả
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")

Mean Squared Error (MSE): 9891066.18029336
Root Mean Squared Error (RMSE): 3145.006546939666
R^2 Score: 0.01781753491506699


## Make prediction in test dataset

In [21]:
test_data_parsed = test_data.map(lambda row: row.split(",")).map(
    lambda parts: (
        parts[0],  # unique trip ID
        [
            float(parts[4]), float(parts[5]),  # pickup_longitude, pickup_latitude
            float(parts[6]), float(parts[7]),  # dropoff_longitude, dropoff_latitude
            float(parts[3])  # passenger_count
        ]
    )
)

In [22]:
test_predictions_rdd = test_data_parsed.map(lambda row: (row[0], predict(tree, row[1])))
test_predictions_df = test_predictions_rdd.toDF(["id", "trip_duration"])

In [ ]:
base_path_result = "/home/tienanh/22120017/Lab03_Spark/src/Regression/Structured_API"

In [24]:
test_predictions_df.show(5, truncate=False)
test_predictions_df.coalesce(1).write.mode("overwrite").parquet(os.path.join(base_path_result, "result"))

+---------+-----------------+
|id       |trip_duration    |
+---------+-----------------+
|id3004672|824.643509031342 |
|id3505355|1429.281770993709|
|id1217141|824.643509031342 |
|id2150126|824.643509031342 |
|id1598245|824.643509031342 |
+---------+-----------------+
only showing top 5 rows

